In [1]:
# Imports PIL module 
import pandas as pd
import spacy
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [2]:
PATH_DATA = './data/data_nlp/'
AD1_FILE = PATH_DATA + 'original_labels.csv'
CHUNK_SIZE = 1000

df = pd.read_csv(AD1_FILE)
df

,entry_id,link_id,parent_id,entry_utc,subreddit,author,body,image,label_date,week,group,sheet_order,annotator_id,level_1,level_2,level_3,strength,highlight
0,exoxn7,t3_exoxn7,NaN,1580652620,badwomensanatomy,doggodone,Do you have the skin of a 80 year old grandma?...,Yes,17-02-2020,1,1,"(1,)",3,Nonmisogynistic,None_of_the_categories,NaN,NaN,NaN
1,exoxn7,t3_exoxn7,NaN,1580652620,badwomensanatomy,doggodone,Do you have the skin of a 80 year old grandma?...,Yes,17-02-2020,1,1,"(1,)",6,Nonmisogynistic,None_of_the_categories,NaN,NaN,NaN
2,exoxn7,t3_exoxn7,NaN,1580652620,badwomensanatomy,doggodone,Do you have the skin of a 80 year old grandma?...,Yes,17-02-2020,1,1,"(1,)",4,Misogynistic,Derogation,Sexual_or_physical_limitations,Nature of the abuse is Implicit,"skin of a 80 year old grandma? Worry no more, ..."
3,fgb3bdv,t3_exoxn7,t3_exoxn7,1580658139,badwomensanatomy,Machaeon,This is taking a grain of truth and extrapolat...,NaN,17-02-2020,1,1,"(1, 1)",3,Nonmisogynistic,None_of_the_categories,NaN,NaN,NaN
4,fgb3bdv,t3_exoxn7,t3_exoxn7,1580658139,badwomensanatomy,Machaeon,This is taking a grain of truth and extrapolat...,NaN,17-02-2020,1,1,"(1, 1)",6,Nonmisogynistic,None_of_the_categories,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15811,fr2pegr,t3_gm99lu,t3_gm99lu,1589841387,AskFeminists,weirddevil,I think that a lot women are held to a higher ...,NaN,18-05-2020,11,2,"(89, 5)",6,Nonmisogynistic,None_of_the_categories,NaN,NaN,NaN
15812,fr2r9me,t3_gm99lu,t3_gm99lu,1589842375,AskFeminists,[deleted],[removed],NaN,18-05-2020,11,2,"(89, 6)",5,Nonmisogynistic,None_of_the_categories,NaN,NaN,NaN
15813,fr2r9me,t3_gm99lu,t3_gm99lu,1589842375,AskFeminists,[deleted],[removed],NaN,18-05-2020,11,2,"(89, 6)",6,Nonmisogynistic,None_of_the_categories,NaN,NaN,NaN
15814,fr326wr,t3_gm99lu,t1_fr2r9me,1589848449,AskFeminists,KaliTheCat,"Please respect our top-level comment rule, whi...",NaN,18-05-2020,11,2,"(89, 6, 1)",6,Nonmisogynistic,None_of_the_categories,NaN,NaN,NaN


In [3]:
df.drop(columns=['entry_id', 'link_id', 'parent_id', 'entry_utc', 'subreddit', 'author',
       'label_date', 'week', 'group', 'sheet_order',
       'annotator_id', 'level_2', 'level_3', 'strength',
       'highlight'], inplace=True)
df

,body,image,level_1
0,Do you have the skin of a 80 year old grandma?...,Yes,Nonmisogynistic
1,Do you have the skin of a 80 year old grandma?...,Yes,Nonmisogynistic
2,Do you have the skin of a 80 year old grandma?...,Yes,Misogynistic
3,This is taking a grain of truth and extrapolat...,NaN,Nonmisogynistic
4,This is taking a grain of truth and extrapolat...,NaN,Nonmisogynistic
...,...,...,...
15811,I think that a lot women are held to a higher ...,NaN,Nonmisogynistic
15812,[removed],NaN,Nonmisogynistic
15813,[removed],NaN,Nonmisogynistic
15814,"Please respect our top-level comment rule, whi...",NaN,Nonmisogynistic


In [4]:
one_hot = pd.get_dummies(df['level_1'])
df['misogynistic'] = one_hot.Misogynistic.values
df

,body,image,level_1,misogynistic
0,Do you have the skin of a 80 year old grandma?...,Yes,Nonmisogynistic,0
1,Do you have the skin of a 80 year old grandma?...,Yes,Nonmisogynistic,0
2,Do you have the skin of a 80 year old grandma?...,Yes,Misogynistic,1
3,This is taking a grain of truth and extrapolat...,NaN,Nonmisogynistic,0
4,This is taking a grain of truth and extrapolat...,NaN,Nonmisogynistic,0
...,...,...,...,...
15811,I think that a lot women are held to a higher ...,NaN,Nonmisogynistic,0
15812,[removed],NaN,Nonmisogynistic,0
15813,[removed],NaN,Nonmisogynistic,0
15814,"Please respect our top-level comment rule, whi...",NaN,Nonmisogynistic,0


In [5]:
df = df[~df['image'].isin(['Yes'])]
df

,body,image,level_1,misogynistic
3,This is taking a grain of truth and extrapolat...,NaN,Nonmisogynistic,0
4,This is taking a grain of truth and extrapolat...,NaN,Nonmisogynistic,0
5,This is taking a grain of truth and extrapolat...,NaN,Nonmisogynistic,0
6,Honestly my favorite thing about this is that ...,NaN,Nonmisogynistic,0
7,Honestly my favorite thing about this is that ...,NaN,Nonmisogynistic,0
...,...,...,...,...
15811,I think that a lot women are held to a higher ...,NaN,Nonmisogynistic,0
15812,[removed],NaN,Nonmisogynistic,0
15813,[removed],NaN,Nonmisogynistic,0
15814,"Please respect our top-level comment rule, whi...",NaN,Nonmisogynistic,0


In [6]:
df.drop(['image', 'level_1'], axis=1, inplace=True)
df

/opt/anaconda3/envs/ada/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,body,misogynistic
3,This is taking a grain of truth and extrapolat...,0
4,This is taking a grain of truth and extrapolat...,0
5,This is taking a grain of truth and extrapolat...,0
6,Honestly my favorite thing about this is that ...,0
7,Honestly my favorite thing about this is that ...,0
...,...,...
15811,I think that a lot women are held to a higher ...,0
15812,[removed],0
15813,[removed],0
15814,"Please respect our top-level comment rule, whi...",0


In [7]:
df.dropna(inplace=True)
df

/opt/anaconda3/envs/ada/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,body,misogynistic
3,This is taking a grain of truth and extrapolat...,0
4,This is taking a grain of truth and extrapolat...,0
5,This is taking a grain of truth and extrapolat...,0
6,Honestly my favorite thing about this is that ...,0
7,Honestly my favorite thing about this is that ...,0
...,...,...
15811,I think that a lot women are held to a higher ...,0
15812,[removed],0
15813,[removed],0
15814,"Please respect our top-level comment rule, whi...",0


In [8]:
df = df[~df['body'].isin(['[removed]'])]

In [9]:
df

,body,misogynistic
3,This is taking a grain of truth and extrapolat...,0
4,This is taking a grain of truth and extrapolat...,0
5,This is taking a grain of truth and extrapolat...,0
6,Honestly my favorite thing about this is that ...,0
7,Honestly my favorite thing about this is that ...,0
...,...,...
15809,I think that due to the traditional view that ...,0
15810,I think that a lot women are held to a higher ...,1
15811,I think that a lot women are held to a higher ...,0
15814,"Please respect our top-level comment rule, whi...",0


In [10]:
df.head(30)

,body,misogynistic
3,This is taking a grain of truth and extrapolat...,0
4,This is taking a grain of truth and extrapolat...,0
5,This is taking a grain of truth and extrapolat...,0
6,Honestly my favorite thing about this is that ...,0
7,Honestly my favorite thing about this is that ...,0
8,Honestly my favorite thing about this is that ...,0
9,Source? Doesnt sound right to me idk,0
10,Source? Doesnt sound right to me idk,0
11,Source? Doesnt sound right to me idk,0
12,"Damn, I saw a movie in which the old woman bat...",1


In [11]:
nlp = spacy.load('en_core_web_sm')
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
import re

def remove_url(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)

def remove_emoji(text):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r"", text)

def text_data_cleaning(sentence):
    sentence = remove_url(sentence)
    sentence = remove_html(sentence)
    sentence = remove_emoji(sentence)
    doc = nlp(sentence)

    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
    
    cleaned_tokens = []
    for token in tokens:
        if token not in nlp.Defaults.stop_words and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [13]:
text_data_cleaning(df.body[3])

['grain',
 'truth',
 'extrapolate',
 'insanity',
 'stay',
 'hydrate',
 'healthy',
 'look',
 'feel',
 'reverse',
 'age',
 'process']

In [14]:
df.body[3]

"This is taking a grain of truth and extrapolating to insanity.\n\nStay hydrated, it's healthy, you'll look and feel better. It will not reverse the aging process though."

In [15]:
remove_emoji(df.body[9])

'Source? Doesnt sound right to me idk'

***TFIDF***

In [16]:
from sklearn.svm import LinearSVC

In [17]:
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning, ngram_range=(1,2))
classifier = LinearSVC()

In [18]:
X = df.body
y = df.misogynistic

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42 )

In [20]:
X_train.shape, X_test.shape

((12160,), (3040,))

In [21]:
X_train[1231]

'Bernie is a commie jew....no wonder the sheep masses love him.\n\nI wonder why the system hates him so much...probably because it\'d be too transparent and people , white people, may start to wake up to our genocide if someone who is openly jewish and anti white was elected...\n\nRemember, they are "white" just like us , goyim'

In [22]:
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])

In [24]:
clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(ngram_range=(1, 2),
                                 tokenizer=<function text_data_cleaning at 0x7fe9db22c9d0>)),
                ('clf', LinearSVC())])

In [25]:
y_pred = clf.predict(X_test)

In [26]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96      2709
           1       0.72      0.58      0.65       331

    accuracy                           0.93      3040
   macro avg       0.84      0.78      0.80      3040
weighted avg       0.93      0.93      0.93      3040



In [27]:
confusion_matrix(y_test, y_pred)

array([[2635,   74],
       [ 138,  193]])

In [28]:
import pickle
filename = './data/data_nlp/classifier2.sav'
pickle.dump(clf, open(filename, 'wb'))

In [29]:
df

,body,misogynistic
3,This is taking a grain of truth and extrapolat...,0
4,This is taking a grain of truth and extrapolat...,0
5,This is taking a grain of truth and extrapolat...,0
6,Honestly my favorite thing about this is that ...,0
7,Honestly my favorite thing about this is that ...,0
...,...,...
15809,I think that due to the traditional view that ...,0
15810,I think that a lot women are held to a higher ...,1
15811,I think that a lot women are held to a higher ...,0
15814,"Please respect our top-level comment rule, whi...",0


***PREDICTIONS***

In [79]:
year = 2016 # available: from 2015 to 2020
PATH_DATA = './data/data_nlp/'
QUOTES_FILE = PATH_DATA + f'quotes-{year}-filtered.json.bz2'
CHUNK_SIZE = 500

reader = pd.read_json(QUOTES_FILE, lines=True, compression='bz2', chunksize=CHUNK_SIZE, typ='frame')

chunks = [] #utile pour plus loins quand on fait le feature extraction
i=0
for chunk in reader:
    df_0 = chunk
    break
df_0.quotation[0]

"[ Malia ] knows what she is going to do. They have a plan for her and her family feels comfortable knowing that it's not something unstructured,"

In [30]:
year = 2020 # available: from 2015 to 2020
PATH_DATA = './data/data_nlp/'
QUOTES_FILE = PATH_DATA + 'metoo_2019_04.json.bz2'

df = pd.read_json(QUOTES_FILE, lines=True, compression='bz2', typ='frame')


In [44]:
df_new = df.iloc[1000:2000]

In [45]:
df_new['Hatred'] = df_new.quotation.apply(lambda quote: clf.predict([quote]))

/var/folders/z2/v95s75vx1flf254b1kb3nfdw0000gn/T/ipykernel_25379/4080023279.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['Hatred'] = df_new.quotation.apply(lambda quote: clf.predict([quote]))


In [46]:
check = df_new[df_new.Hatred==1]


In [50]:
check.quotation.values

array(["Massive bitch aka feminist woman you can't control. Diddums."],
      dtype=object)

In [51]:
df['Hatred'] = df.quotation.apply(lambda quote: clf.predict([quote]))

In [60]:
check = df[df.Hatred==1]
check.shape

(21, 10)

In [59]:
for quote in check.quotation:
    print(quote)

Sexual Duality and Sexual Multiplicity in Western Art.
Somebody HIV + should rape this bitch.
Massive bitch aka feminist woman you can't control. Diddums.
I think women are more than smart enough to know that (a) Joe Biden is who he is and hugs everybody, (b) this was minor compared to most sexual harassment.
The people who publicly talk about sex and sexuality are super-predominantly women. The way that men have been programmed how to talk about sex, is by bragging and objectifying women. The idea that sexual sensitivity is emasculating is so sad. It's tragic for men and it's tragic for women and that's not healthy for a romantic or sexual relationship. Spoiler alert: men are actually sexually repressed too and women need to start accepting that. There is a faction of the feminist movement that's man-hating and I find that off-putting.
carry out sexual acts on themselves
Feminists have long argued that traditional gender roles harm women... It's also possible, though, that received ex

In [158]:
s = df_new[df_new.Hatred == 1].quotation.iloc[3]

In [159]:
def split(text, n=100):
    return []

'Thankfully, with places like Optimism Place, we have supports put in place for people who are involved, or who are victims or survivors of domestic violence and we have the ability to help those people through those problems -- not only with the survivors, but with those families as well in helping to prevent any further problems from happening.'

In [164]:
s_s = [s[100*i:100*(i+1)] for i in range(len(s)//100)] + [s[100*(len(s)//100):]];s_s

['Thankfully, with places like Optimism Place, we have supports put in place for people who are involv',
 'ed, or who are victims or survivors of domestic violence and we have the ability to help those peopl',
 'e through those problems -- not only with the survivors, but with those families as well in helping ',
 'to prevent any further problems from happening.']

In [167]:
for s in s_s:
    clf.predict([s])


In [168]:
clf.predict(s_s)

array([0, 1, 1, 0])

In [61]:
df.shape

(9914, 10)